# Install Packages dibawah terlebih dahulu

In [17]:
import pyderman as driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
import gspread
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np
import time
import os
import re
driver = webdriver.Chrome()


Masuk ke website evisum - Masukkan USERNAME DAN PASSWORD SERTA SELESAIKAN CAPTCHA

![Halaman Evsium](img/evisum.png)

In [22]:
pkb = pd.read_excel("data/pkb_kec.xlsx")
pkb['NIP'] = pkb['NIP'].astype(str)
pkb = pkb[['Kabupaten', 'Kecamatan', 'NIP', 'Nama Lengkap']]
pkb

,Kabupaten,Kecamatan,NIP,Nama Lengkap
0,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI
1,PASANGKAYU,BAMBALAMOTU,199702082023212029,"FITRI DAYANTI, S.E"
2,PASANGKAYU,BAMBALAMOTU,197009292012122002,NUR AYNY
3,PASANGKAYU,BAMBALAMOTU,199707072023212027,"SARI ASTUTI, S.Sos"
4,PASANGKAYU,BAMBALAMOTU,198512032011011007,"HASANUDDIN, SE"
...,...,...,...,...
422,MAMUJU TENGAH,KAROSSA,198702252022212002,"HANA PRATIWI ARSYAD, S.ST"
423,MAMUJU TENGAH,KAROSSA,199309132022212003,"IVA MUKRIMAH, S.Tr.Keb"
424,MAMUJU TENGAH,KAROSSA,198206202022211002,"ABDULLAH, A.Md.Kep"
425,MAMUJU TENGAH,KAROSSA,199501012023212066,"NURHASIA L., A.Md.Keb"


In [23]:
# Tentukan bulan dan tahun yang diinginkan
year = 2024
month = 8

# Generate semua tanggal pada bulan tertentu
date_range = pd.date_range(start=f'{year}-{month:02d}-01', end=f'{year}-{month:02d}-31')

# Filter untuk menghapus hari Sabtu (5) dan Minggu (6)
weekdays = date_range[~date_range.weekday.isin([5, 6])]

# Format tanggal menjadi dd-mm-yyyy
formatted_dates = weekdays.strftime('%d-%m-%Y')
print(formatted_dates)

Index(['01-08-2024', '02-08-2024', '05-08-2024', '06-08-2024', '07-08-2024',
       '08-08-2024', '09-08-2024', '12-08-2024', '13-08-2024', '14-08-2024',
       '15-08-2024', '16-08-2024', '19-08-2024', '20-08-2024', '21-08-2024',
       '22-08-2024', '23-08-2024', '26-08-2024', '27-08-2024', '28-08-2024',
       '29-08-2024', '30-08-2024'],
      dtype='object')


In [24]:
formatted_dates = formatted_dates[0:11] #mengambil tanggal 1-16

In [25]:
#membuat dataframe kosong
presensi_pkb = pd.DataFrame(columns = ['Kabupaten', 'Kecamatan', 'NIP', 'Nama',
                               'Tanggal', 'lat1', 'lon1', 'lat2', 'lon2'])
presensi_pkb

,Kabupaten,Kecamatan,NIP,Nama,Tanggal,lat1,lon1,lat2,lon2


In [ ]:
for i in range(0, 426): #jumlah PKB
    for j in range(0, len(formatted_dates)): #jumlah hari
        print(str(i), " - ", str(j))

        link = "https://evisum4.bkkbn.go.id/showMapPresensi?nip=" + pkb.iloc[i, 2] + "&tgl=" + formatted_dates[j]

        driver.get(link)
        sleep(1)

        script_element = driver.find_element(By.XPATH, "/html/body/script")

        # Mengambil teks dari elemen <script>
        script_text = script_element.get_attribute('innerHTML')

        # Regex untuk mengekstrak koordinat dan URL gambar
        coordinates = re.findall(r'L\.marker\(\[(.*?)\]\)', script_text)

        if len(coordinates) > 1:
            coordinates_datang = coordinates[0]
            lat1, lon1 = coordinates_datang.split(', ')

            cordinates_pulang = coordinates[1]
            lat2, lon2 = cordinates_pulang.split(', ')
        elif len(coordinates) == 1:
            coordinates_datang = coordinates[0]
            lat1, lon1 = coordinates_datang.split(', ')
            lat2, lon2 = [0, 0]
        else:
            lat1, lon1, lat2, lon2 = [0, 0, 0, 0]

        # Memisahkan latitude dan longitude
        # if len(coordinates[0]) > 0:
        #     coordinates_datang = coordinates[0]
        #     lat1, lon1 = coordinates[0].split(', ')
        # else:
        #     lat1 = 0
        #     lon1 = 0

        # # Memisahkan latitude dan longitude
        # if len(coordinates[1]) > 0:
        #     cordinates_pulang = coordinates[1]
        #     lat2, lon2 = coordinates[1].split(', ')
        # else:
        #     lat2 = 0
        #     lon2 = 0

        presensi_pkb_sementara = pd.DataFrame({'Kabupaten': pkb.iloc[i,0],
                    'Kecamatan': pkb.iloc[i,1],
                    'NIP': pkb.iloc[i,2],
                    'Nama': pkb.iloc[i,3],
                    'Tanggal': formatted_dates[j],
                    'lat1': lat1,
                    'lon1': lon1,
                    'lat2': lat2,
                    'lon2': lon2}, index=[0])
        
        presensi_pkb = pd.concat([presensi_pkb, presensi_pkb_sementara], ignore_index=True)
        print('Berhasil')
print("Selesai")
        

In [28]:
presensi_pkb

,Kabupaten,Kecamatan,NIP,Nama,Tanggal,lat1,lon1,lat2,lon2
0,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,01-08-2024,-1.1753366666667,119.37623833333,-1.174635,119.376505
1,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,02-08-2024,-1.17458666666667,119.376376666667,-1.1745957,119.3767491
2,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,05-08-2024,-2.6722541,118.8908479,0,0
3,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,06-08-2024,-1.1762867,119.3695666,-1.1741861,119.3756639
4,PASANGKAYU,BAMBALAMOTU,196911152006042008,KUR`ANI,07-08-2024,-1.1746316666667,119.37655833333,-1.1764816666667,119.369485
...,...,...,...,...,...,...,...,...,...
4692,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",09-08-2024,-1.7812683333333,119.395975,-1.7942033333333,119.40647166667
4693,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",12-08-2024,-1.7940083333333,119.40567833333,-1.7837033333333,119.390685
4694,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",13-08-2024,-1.7804366666667,119.39209,-1.790367,119.3956205
4695,MAMUJU TENGAH,KAROSSA,197206011994031006,"NASARANI, AMD.Kep.",14-08-2024,-1.7836566666667,119.39017666667,-1.787825,119.3883638


In [27]:
presensi_pkb.to_excel("data/agustus_presensi.xlsx")

In [ ]:
# batas bawah